# About Dataset

The dataset contains EEG signals from 11 subjects with labels of alert and drowsy. It can be opened with Matlab. We extracted the data for our own research purpose from another public dataset:

Cao, Z., et al., Multi-channel EEG recordings during a sustained-attention driving task. Scientific data, 2019. 6(1): p. 1-8.

If you find the dataset useful, please give credits to their works.

The details on how the data were extracted are described in our paper:

"Jian Cui, Zirui Lan, Yisi Liu, Ruilin Li, Fan Li, Olga Sourina, Wolfgang Müller-Wittig, A Compact and Interpretable Convolutional Neural Network for Cross-Subject Driver Drowsiness Detection from Single-Channel EEG, Methods, 2021, ISSN 1046-2023, https://doi.org/10.1016/j.ymeth.2021.04.017."

The codes of the paper above are accessible from:

https://github.com/cuijiancorbin/A-Compact-and-Interpretable-Convolutional-Neural-Network-for-Single-Channel-EEG

The data file contains 3 variables and they are EEGsample, substate and subindex.

"EEGsample" contains 2022 EEG samples of size 20x384 from 11 subjects. Each sample is a 3s EEG data with 128Hz from 30 EEG channels.
"subindex" is an array of 2022x1. It contains the subject indexes from 1-11 corresponding to each EEG sample.
"substate" is an array of 2022x1. It contains the labels of the samples. 0 corresponds to the alert state and 1 correspond to the drowsy state.

The unbalanced version of this dataset is accessible from:
https://figshare.com/articles/dataset/EEG_driver_drowsiness_dataset_unbalanced_/16586957

# Importing Libraires 

In [ ]:
import scipy.io
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

# Loading Dataset

In [ ]:
import scipy.io

# Load the data from the provided .mat file
file_path = 'EEG driver drowsiness dataset.mat'
mat_data = scipy.io.loadmat(file_path)

# Inspecting the keys and structure of the loaded data
mat_data.keys(), {key: type(mat_data[key]) for key in mat_data.keys()}

# Inspecting the shape and content of the EEGsample, subindex, and substate arrays

In [ ]:
# Inspecting the shape and content of the EEGsample, subindex, and substate arrays
eeg_sample_shape = mat_data['EEGsample'].shape
subindex_shape = mat_data['subindex'].shape
substate_shape = mat_data['substate'].shape

eeg_sample_shape, subindex_shape, substate_shape

In [ ]:
print(mat_data)

#### The initial exploration of the EEG dataset reveals the following details:

#### EEG Samples (EEGsample):

  * The dataset contains 2022 EEG samples.
  * Each EEG sample is from 30 channels.
  * Each channel has 384 data points, corresponding to a 3-second EEG recording at a sampling rate of 128Hz.

#### Subject States (substate):

  * There are two unique states: 0 representing the alert state and 1 representing the drowsy state.
  * Each state has 1011 samples, indicating a balanced dataset with respect to the two states.

#### Subject Indexes (subindex):

  * There are 11 unique subjects in the dataset (labeled 1 to 11).
  * The distribution of samples across subjects varies, ranging from a minimum of 102 samples to a maximum of 314 samples per subject.

# Analyzing EEG Data: Determining Sample Shape, Channel Count, and Duration

In [ ]:
# Access the EEG data and other relevant information
data = mat_data
eeg_samples = data['EEGsample']
subindex = data['subindex']
substate = data['substate']


num_samples, num_channels, num_time_points = eeg_sample_shape
subject_indexes = mat_data['subindex'].flatten()
labels = mat_data['substate'].flatten()

# Check for missing values

In [ ]:
# Check for missing values
missing_values = np.isnan(eeg_samples).sum()
if missing_values == 0:
    print("No missing values in the EEG data.")
else:
    print("Number of missing values:", missing_values)

### Calculate the duration of each sample in seconds

In [ ]:
# Calculate the duration of each sample in seconds
sampling_rate = 128  # Hz
sample_duration = num_time_points / sampling_rate

In [ ]:
print("Number of subjects:", len(np.unique(subject_indexes)))
print("EEGsample shape:", eeg_sample_shape)
print("Number of Samples:", num_samples)
print("Number of Channels:", num_channels)
print("Number of Time Points:", num_time_points)
print("Sample Duration (seconds):", sample_duration)

unique_labels, label_counts = np.unique(labels, return_counts=True)
print("Unique labels:", unique_labels)
print("Label counts:", label_counts) 

# Data Visualization

## Distribution of Subindex and Substate Values

In [ ]:
# Get unique values and their counts for subindex and substate
unique_subindex, counts_subindex = np.unique(mat_data['subindex'], return_counts=True)
unique_substate, counts_substate = np.unique(mat_data['substate'], return_counts=True)

# Set up subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot histogram for subindex
axes[0].bar(unique_subindex, counts_subindex, align='center', alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Frequency of Subindex Values')
axes[0].set_xlabel('Subindex Values')
axes[0].set_ylabel('Frequency')
axes[0].grid(True, linestyle='--', alpha=0.7)

# Add count annotations on top of the bars for subindex
for x, y in zip(unique_subindex, counts_subindex):
    axes[0].text(x, y + 0.1, str(y), ha='center', va='bottom')

# Plot histogram for substate
axes[1].bar(unique_substate, counts_substate, align='center', alpha=0.7, color='lightcoral', edgecolor='black')
axes[1].set_title('Frequency of Substate Values')
axes[1].set_xlabel('Substate Values')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, linestyle='--', alpha=0.7)

# Add count annotations on top of the bars for substate
for x, y in zip(unique_substate, counts_substate):
    axes[1].text(x, y + 0.1, str(y), ha='center', va='bottom')

# Adjust layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()

## Analysis of Substate Distribution Across Subindices

In [ ]:
# Group by subindex and count occurrences of 0 and 1 in substate
subindex_values, substate_counts = np.unique(mat_data['subindex'], return_counts=True)
substate_0_counts = []
substate_1_counts = []

for subindex_value in subindex_values:
    substate_values_for_subindex = mat_data['substate'][mat_data['subindex'] == subindex_value]
    substate_0_counts.append(np.sum(substate_values_for_subindex == 0))
    substate_1_counts.append(np.sum(substate_values_for_subindex == 1))

# Set up the bar chart
fig, ax = plt.subplots(figsize=(10, 6))

bar_width = 0.35
bar_positions_0 = np.arange(len(subindex_values))
bar_positions_1 = bar_positions_0 + bar_width

ax.bar(bar_positions_0, substate_0_counts, width=bar_width, label='Substate 0', alpha=0.7, color='skyblue', edgecolor='black')
ax.bar(bar_positions_1, substate_1_counts, width=bar_width, label='Substate 1', alpha=0.7, color='lightcoral', edgecolor='black')

# Add count annotations on top of each bar
for x, y in zip(bar_positions_0, substate_0_counts):
    ax.text(x, y + 0.1, str(y), ha='center', va='bottom')

for x, y in zip(bar_positions_1, substate_1_counts):
    ax.text(x, y + 0.1, str(y), ha='center', va='bottom')

ax.set_xticks(bar_positions_0 + bar_width / 2)
ax.set_xticklabels(subindex_values)
ax.set_xlabel('Subindex Values')
ax.set_ylabel('Frequency')
ax.set_title('Frequency of Substate Values for Each Subindex')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.7)

plt.show()


# Basic Statistical Overview of the EEG data

## State-specific Analysis: Mean and Standard Deviation of EEG Channels for 'Alert' and 'Drowsy' States

In [ ]:
# Extracting individual components from the dataset
EEGsamples = data['EEGsample']
substates = data['substate'].ravel()  # Flatten the array

In [ ]:
# Calculating means and standard deviations for each channel in both states

#Alert States
mean_alert = np.mean(EEGsamples[substates == 0], axis=(0, 2))
std_alert = np.std(EEGsamples[substates == 0], axis=(0, 2))

#Drowsy States
mean_drowsy = np.mean(EEGsamples[substates == 1], axis=(0, 2))
std_drowsy = np.std(EEGsamples[substates == 1], axis=(0, 2))

In [ ]:
# Creating a DataFrame for easy viewing
stats_df = pd.DataFrame({
    'Channel': range(1, 31),
    'Mean_Alert': mean_alert,
    'Std_Alert': std_alert,
    'Mean_Drowsy': mean_drowsy,
    'Std_Drowsy': std_drowsy
})

In [ ]:
stats_df  

## EEG Channel Statistics Across Subjects and Timepoints

In [ ]:
# Compute basic statistics for each EEG channel
mean_values = np.mean(eeg_samples, axis=(0, 2))  # Compute mean along subjects and timepoints
std_values = np.std(eeg_samples, axis=(0, 2))    # Compute standard deviation along subjects and timepoints
min_values = np.min(eeg_samples, axis=(0, 2))    # Compute minimum along subjects and timepoints
max_values = np.max(eeg_samples, axis=(0, 2))    # Compute maximum along subjects and timepoints

# Display basic statistics for each EEG channel
for channel_index in range(num_channels):
    print("Channel {}: Mean: {:.4f}, Std: {:.4f}, Min: {:.4f}, Max: {:.4f}".format(
        channel_index + 1, mean_values[channel_index], std_values[channel_index],
        min_values[channel_index], max_values[channel_index]
    ))

# Preprocessing

In [ ]:
import scipy.io
import numpy as np

# Load the dataset
file_path = 'EEG driver drowsiness dataset.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant variables
EEGsample = data['EEGsample']
substate = data['substate']
subindex = data['subindex']

# Initial data exploration
EEGsample_shape = EEGsample.shape
substate_unique, substate_counts = np.unique(substate, return_counts=True)
subindex_unique, subindex_counts = np.unique(subindex, return_counts=True)

EEGsample_shape, substate_unique, substate_counts, subindex_unique, subindex_counts



In [ ]:
from scipy.signal import butter, filtfilt

# Function to apply a bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_data = filtfilt(b, a, data, axis=2)
    return filtered_data

# Apply the bandpass filter (1-50 Hz) to the EEG data
fs = 128  # Sampling rate
lowcut = 1.0  # Low frequency limit
highcut = 50.0  # High frequency limit
EEGsample_filtered = bandpass_filter(EEGsample, lowcut, highcut, fs)

# Artifact Removal: Identify and mark samples with excessive amplitude
# (Defining a simple threshold based approach)
amplitude_threshold = 100  # Threshold for identifying artifacts, needs tuning based on the data
artifact_mask = np.max(np.abs(EEGsample_filtered), axis=(1, 2)) > amplitude_threshold

# Normalize the EEG data
EEGsample_normalized = EEGsample_filtered / np.max(np.abs(EEGsample_filtered))

# Summarizing the preprocessing steps
preprocessing_summary = {
    "Filtered_Data_Shape": EEGsample_filtered.shape,
    "Artifacts_Detected": np.sum(artifact_mask),
    "Normalized_Data_Shape": EEGsample_normalized.shape
}

preprocessing_summary



#  Feature extraction

In [ ]:
from scipy.stats import skew, kurtosis

# Function to calculate Hjorth parameters
def hjorth_parameters(data):
    first_deriv = np.diff(data, axis=2)
    second_deriv = np.diff(first_deriv, axis=2)

    activity = np.var(data, axis=2)
    mobility = np.sqrt(np.var(first_deriv, axis=2) / activity)
    complexity = np.sqrt(np.var(second_deriv, axis=2) / np.var(first_deriv, axis=2)) / mobility

    return activity, mobility, complexity

# Time-domain feature extraction
mean_values = np.mean(EEGsample_normalized, axis=2)
std_dev_values = np.std(EEGsample_normalized, axis=2)
skewness_values = skew(EEGsample_normalized, axis=2)
kurtosis_values = kurtosis(EEGsample_normalized, axis=2)

activity, mobility, complexity = hjorth_parameters(EEGsample_normalized)

# Summarize the extracted features
time_domain_features = {
    "Mean_Values": mean_values.shape,
    "Standard_Deviation": std_dev_values.shape,
    "Skewness": skewness_values.shape,
    "Kurtosis": kurtosis_values.shape,
    "Activity": activity.shape,
    "Mobility": mobility.shape,
    "Complexity": complexity.shape
}

time_domain_features



In [ ]:
from scipy.signal import welch

# Function to calculate Power Spectral Density (PSD) and Band Power Ratios
def calculate_psd_and_band_power(data, fs):
    # Define frequency bands
    bands = {
        'Delta': (1, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30)
    }

    # Calculate PSD using Welch's method
    freqs, psd = welch(data, fs=fs, axis=2)

    # Extracting band power and ratios
    band_power = {band: np.trapz(psd[:, :, (freqs >= low) & (freqs <= high)], axis=2)
                  for band, (low, high) in bands.items()}

    return band_power, freqs, psd

# Frequency-domain feature extraction
band_power, freqs, psd = calculate_psd_and_band_power(EEGsample_normalized, fs)

# Summarize the extracted features
frequency_domain_features = {
    "Delta_Band_Power": band_power['Delta'].shape,
    "Theta_Band_Power": band_power['Theta'].shape,
    "Alpha_Band_Power": band_power['Alpha'].shape,
    "Beta_Band_Power": band_power['Beta'].shape,
    "PSD_Shape": psd.shape
}

frequency_domain_features



# The time-frequency domain feature extraction using Wavelet Transform 

In [ ]:
import pywt
# Function to calculate Wavelet Transform features correctly
def calculate_wavelet_features_corrected(data, wavelet_name='db4'):
    # List to store the wavelet features
    wavelet_features = []

    for sample in data:
        sample_features = []
        for channel in sample:
            # Wavelet decomposition
            coeffs = pywt.wavedec(channel, wavelet_name, level=5)
            # Calculate energy for each level
            energy = np.array([np.sum(np.square(coeff)) for coeff in coeffs])
            # Calculate entropy for each level
            entropy = np.array([-np.sum(coeff * np.log2(np.abs(coeff) + 1e-12)) for coeff in coeffs])
            # Combine energy and entropy
            sample_features.append(np.concatenate([energy, entropy]))
        wavelet_features.append(np.array(sample_features))
    
    return np.array(wavelet_features)

# Recalculate wavelet features for the EEG data
wavelet_features_corrected = calculate_wavelet_features_corrected(EEGsample_normalized)

# Summarize the extracted features
time_frequency_domain_features_corrected = {
    "Wavelet_Features_Shape": wavelet_features_corrected.shape
}

time_frequency_domain_features_corrected



# Exploratory data analysis (EDA) of the EEG dataset


For the exploratory data analysis (EDA) of the EEG dataset, we'll focus on uncovering patterns and insights from the extracted features. The EDA will be structured as follows:

Descriptive Statistics:

Analyze the basic statistics (mean, standard deviation, etc.) of the extracted features across different states (alert and drowsy) and subjects.
Feature Distribution Analysis:

Examine the distributions of key features like band powers and Hjorth parameters for different states and subjects.
Use histograms, boxplots, or violin plots for visualization.
Correlation Analysis:

Assess the correlation between different features to identify any strong relationships or redundancies.
State-wise Comparison:

Compare the features for alert and drowsy states to see if any features significantly differ between these states.
This can be done using statistical tests like t-tests or ANOVA for numerical features.
Subject Variability Analysis:

Explore how features vary across different subjects to understand inter-subject variability.
Investigate if any subject-specific patterns emerge.
Time-Frequency Feature Analysis:

Analyze wavelet-based features to see how energy and entropy vary across different time-frequency scales and states.
Visualization:

Create visual representations of the data and features to better understand the underlying patterns.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convert features and labels into a DataFrame for easier analysis
features_df = pd.DataFrame({
    'Subject': subindex.flatten(),
    'State': substate.flatten(),
    'Delta_Power': band_power['Delta'].mean(axis=1),
    'Theta_Power': band_power['Theta'].mean(axis=1),
    'Alpha_Power': band_power['Alpha'].mean(axis=1),
    'Beta_Power': band_power['Beta'].mean(axis=1),
    'Activity': activity.mean(axis=1),
    'Mobility': mobility.mean(axis=1),
    'Complexity': complexity.mean(axis=1)
})

# Descriptive statistics
desc_stats = features_df.groupby('State').describe()

# Distribution analysis: Band Powers and Hjorth Parameters
plt.figure(figsize=(15, 10))
for i, feature in enumerate(['Delta_Power', 'Theta_Power', 'Alpha_Power', 'Beta_Power', 'Activity', 'Mobility', 'Complexity'], 1):
    plt.subplot(3, 3, i)
    sns.boxplot(x='State', y=feature, data=features_df)
    plt.title(feature)

plt.tight_layout()
plt.show()

desc_stats



# Correlation Analysis

In [ ]:
# Calculating the correlation matrix
correlation_matrix = features_df.drop('Subject', axis=1).corr()

# Plotting the correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix of EEG Features")
plt.show()

correlation_matrix



# State-wise Comparison Using T-Tests

In [ ]:
from scipy.stats import ttest_ind

# Function to perform t-test for each feature between alert and drowsy states
def perform_t_tests(df, features):
    t_test_results = {}
    for feature in features:
        alert_values = df[df['State'] == 0][feature]
        drowsy_values = df[df['State'] == 1][feature]
        t_stat, p_value = ttest_ind(alert_values, drowsy_values, equal_var=False)
        t_test_results[feature] = {'t_stat': t_stat, 'p_value': p_value}
    return t_test_results

# Features to perform t-test on
features_for_t_test = ['Delta_Power', 'Theta_Power', 'Alpha_Power', 'Beta_Power', 'Activity', 'Mobility', 'Complexity']

# Performing the t-tests
t_test_results = perform_t_tests(features_df, features_for_t_test)

t_test_results



# subject variability analysis

In [ ]:
# Visualization of feature variability across subjects
plt.figure(figsize=(20, 15))

# Selecting a subset of features for visualization
features_to_visualize = ['Delta_Power', 'Theta_Power', 'Alpha_Power', 'Beta_Power', 'Activity', 'Mobility', 'Complexity']

# Plotting boxplots for each feature across subjects
for i, feature in enumerate(features_to_visualize, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(x='Subject', y=feature, data=features_df)
    plt.title(feature)

plt.tight_layout()
plt.show()

# Note: Due to the large number of subjects, the individual boxplots might be closely packed. The aim is to observe overall trends and variations.


# Detailed Subject-State Analysis

In [ ]:
# Visualization of feature comparison between states for each subject
plt.figure(figsize=(20, 30))

# Looping through each feature for visualization
for i, feature in enumerate(features_to_visualize, 1):
    plt.subplot(len(features_to_visualize), 1, i)
    sns.boxplot(x='Subject', y=feature, hue='State', data=features_df)
    plt.title(f"{feature} Comparison Between Alert and Drowsy States Across Subjects")
    plt.legend(title='State', loc='upper right')

plt.tight_layout()
plt.show()

# Note: Due to the complexity of the dataset and the number of subjects, these plots provide a high-level overview. Detailed analysis might require focusing on specific subjects or features.


# Focusing on specific subjects or features 

In [ ]:
# Selecting a subset of subjects for detailed analysis
# Choosing subjects who showed significant variability or unique patterns in previous analyses
selected_subjects = [1, 5, 11]  # Example subjects, can be adjusted based on specific interests

# Focusing on a couple of features for detailed analysis
selected_features = ['Alpha_Power', 'Mobility']  # Features with significant differences or high variability

# Filtering the data for selected subjects and features
filtered_df = features_df[features_df['Subject'].isin(selected_subjects)]

# Visualization of the selected features for the selected subjects
plt.figure(figsize=(15, 10))

for i, feature in enumerate(selected_features, 1):
    plt.subplot(len(selected_features), 1, i)
    sns.lineplot(x='Subject', y=feature, hue='State', data=filtered_df, marker='o')
    plt.title(f"{feature} Across Selected Subjects")
    plt.legend(title='State', loc='upper right')

plt.tight_layout()
plt.show()

# Note: The line plots provide a comparative view of the selected features across the chosen subjects and states.


# Visualization of EEG waveforms

In [ ]:
# Selecting all channels for visualization
all_channels = list(range(1, 31))

# Creating plots
fig, axes = plt.subplots(nrows=len(all_channels), ncols=2, figsize=(15, 5 * len(all_channels)))
fig.suptitle('EEG Waveforms Comparison: Alert vs Drowsy States', fontsize=16)

# Plotting for each channel
for i, channel in enumerate(all_channels):
    
    
    # Alert state
    axes[i, 0].plot(EEGsamples[substates == 0][:, channel - 1, :].mean(axis=0))
    axes[i, 0].set_title(f'Channel {channel} - Alert State')
    axes[i, 0].set_xlabel('Time Points')
    axes[i, 0].set_ylabel('EEG Signal')
    axes[i, 0].grid(True)  # Add grid lines

    
    # Drowsy state
    axes[i, 1].plot(EEGsamples[substates == 1][:, channel - 1, :].mean(axis=0))
    axes[i, 1].set_title(f'Channel {channel} - Drowsy State')
    axes[i, 1].set_xlabel('Time Points')
    axes[i, 1].grid(True)  # Add grid lines

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have the EEGsamples and substates defined

# Selecting all channels for visualization
all_channels = list(range(1, 31))

# Creating plots
fig, axes = plt.subplots(nrows=len(all_channels), ncols=1, figsize=(10, 3 * len(all_channels)))
fig.suptitle('EEG Waveforms Comparison: Alert vs Drowsy States', fontsize=16)

# Plotting for each channel
for i, channel in enumerate(all_channels):
    # Alert state
    alert_mean = EEGsamples[substates == 0][:, channel - 1, :].mean(axis=0)
    axes[i].plot(alert_mean, label='Alert', alpha=0.7)

    # Drowsy state
    drowsy_mean = EEGsamples[substates == 1][:, channel - 1, :].mean(axis=0)
    axes[i].plot(drowsy_mean, label='Drowsy', alpha=0.7)

    axes[i].set_title(f'Channel {channel}')
    axes[i].set_xlabel('Time Points')
    axes[i].set_ylabel('EEG Signal')
    axes[i].legend()

    # Add grid
    axes[i].grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# Function to plot EEG signals for a given sample
def plot_eeg_sample(eeg_data, sample_index, title):
    plt.figure(figsize=(15, 6))
    for i in range(eeg_data.shape[1]):
        plt.plot(eeg_data[sample_index, i, :], label=f'Channel {i+1}')
    plt.title(title)
    plt.xlabel('Time Points')
    plt.ylabel('EEG Signal Amplitude')
    plt.legend()
    plt.show()

# Find the index of the first occurrence of 0 (alert state)
sample_alert_index = list(substate).index(0)

# Find the index of the first occurrence of 1 (drowsy state)
sample_drowsy_index = list(substate).index(1)

# Plot EEG samples for alert and drowsy states
plot_eeg_sample(eeg_samples, sample_alert_index, 'EEG Sample - Alert State')
plot_eeg_sample(eeg_samples, sample_drowsy_index, 'EEG Sample - Drowsy State')




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_subjects = 11  # Assuming you have 11 subjects
num_channels = 30  # Assuming you have 30 channels

for subject in range(num_subjects):
    # Create a figure for each subject
    fig, axes = plt.subplots(nrows=num_channels, ncols=1, figsize=(10, 3 * num_channels))
    fig.suptitle(f'EEG Data for Subject {subject + 1}', fontsize=16)

    for channel in range(num_channels):
        # Extract data for the current channel and subject
        channel_data = EEGsamples[subject, channel, :]

        # Plotting
        axes[channel].plot(channel_data, label=f'Channel {channel + 1}')
        axes[channel].set_title(f'Channel {channel + 1}')
        axes[channel].set_xlabel('Time Points')
        axes[channel].set_ylabel('EEG Signal')
        axes[channel].legend()
        axes[channel].grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np

# Summary statistics for the entire EEGsamples dataset
mean_value = np.mean(EEGsamples)
std_value = np.std(EEGsamples)
min_value = np.min(EEGsamples)
max_value = np.max(EEGsamples)

print(f"Overall Mean: {mean_value}, Std: {std_value}, Min: {min_value}, Max: {max_value}")


In [ ]:
import matplotlib.pyplot as plt

# Example: Subject 1, Channel 1
subject = 0
channel = 0
plt.plot(EEGsamples[subject, channel, :])
plt.title(f'EEG Data for Subject {subject+1}, Channel {channel+1}')
plt.xlabel('Time Points')
plt.ylabel('EEG Signal')
plt.show()


In [ ]:
import pandas as pd

# Display data for Subject 1 across all channels and first 10 time points
data_slice = EEGsamples[subject, :, :10]
df = pd.DataFrame(data_slice)
print(df)


In [ ]:
# Print the value for Subject 1, Channel 1, at the first time point
print("Specific value:", EEGsamples[subject, channel, 0])


In [ ]:
https://www.kaggle.com/code/tenebris97/emotions-eeg-lstm-gru-dnn-98-44

In [ ]:
https://www.kaggle.com/code/tenebris97/gridsearch-shap-lstm-dnn